# I'm a robot: Breaking ReCAPTCHA

## Introduction

The main purpose of a CAPTCHA system is to prevent automated system access by computer programs and other bots. There are various versions of CAPTCH with distorted images as shown below.

[![CAPTCHAs.png](https://s18.postimg.org/4o0nc6bbt/CAPTCHAs.png)](https://postimg.org/image/v9367qdp1/)

Google has been actively trying to prevent robots from breaking their systems, and their reCAPTACHA system has gained wide popularity all over the world. Hence, we are particularly interested in the reCAPTCHA system in this project. ReCAPTCHA currently features a "I am not a robot" check box, and we may pass the check even without solving a puzzle. Behind this checkbox is a risk analysis engine, and robots need to behave like a human to cheat this engine.

[![Not_a_robot.png](https://s18.postimg.org/ql3wbrnah/Not_a_robot.png)](https://postimg.org/image/q8ci5l50l/)

When we cannot pass the check directly, we will still need to solve the puzzle, and this is the main focus of this project. ReCAPTCHAs version 1 uses text image scanned from accent books, and road signs directly snapped from Google Street View. Incorporating two different style text images in one reCAPTCHA image makes existing algorithms targeting CAPTCHA incorporated single scheme powerless. In addition, reCAPTCHA often added blurry scanned images makes it difficult to decipher even for human users. With a series of image processing and machine learning algorithms, We want to find out how difficult for a robot to solve the puzzle and what kind of distortion is most effective. 

[![img593.jpg](https://s22.postimg.org/c1uyzaptt/img593.jpg)](https://postimg.org/image/f8piixa9p/)

## Real-world data gathering

As Google provides easily set-up reCAPTCHA API, there is no need to crawl reCAPTCHA images from the actual website that uses Google reCAPTCHA service, real-world reCAPTCHA data can be collected easily by integrating reCAPTCHA service with our own server. Simply sign up a reCAPTCHA API key with our own Gmail address, we can request legit reCAPTCHA images on the fly.

## ReCAPTCHA preprocessing

The text image data requires several stages of processing before it can be fed to our deep learning system.
The raw reCAPTCHA image we get from Google Service consists two separate text images, which make it hard to process at once. Dark background text images are often combined with light background ones, which put an extra layer of difficulty during processing.

[![img665.jpg](https://s18.postimg.org/vy32u99ah/img665.jpg)](https://postimg.org/image/o5cf2a3b9/)

First, we split the image so that we can process each half individually, then converts the images into grayscale as all we only care about segmenting the text. The each image is purposely skewed, therefore the next step to recover the image to the proper angle.

[![img665_split0.jpg](https://s22.postimg.org/huac2hg01/img665_split0.jpg)](https://postimg.org/image/ijt4eugjh/)[![img665_split1.jpg](https://s11.postimg.org/4y0q4lh4z/img665_split1.jpg)](https://postimg.org/image/gn4psk83j/)

## Slicing text image

The major challenge we are facing during the preprocessing stage is how to slice a text image. We have experimented several approaches, and we are still trying a new approach to yield better results.  

### Experimental approach I

First, we have tried to utilize to openCV’s threshold API to convert the text image into a binary image then find the contour of each character, which helps us the construct bounding box of each character. However, this approach relies on the high contrast of the image to separate text from background, which is not always the case. Besides, as the images are mixed of dark background ones and light background ones, it becomes hard to set a threshold that works for all images. The main issue comes when the difference between text color and background are so subtle that simply apply such binary conversion would make the text blend in the background.

[![29.jpg](https://s21.postimg.org/40zpuwhk7/image.jpg)](https://postimg.org/image/539wdg0df/)

### Experimental approach II

The second approach we have tried is to linearly scan the image. However, reCaptcha has various fonts, some are relatively round, while some are slim. For slim fonts, two characters may sit in on window, which potentially confuses the model. Also, it works for text images with a noticeable distance between characters, while sometimes there is no spe

[![24.jpg](https://s21.postimg.org/618rc7ihj/image.jpg)](https://postimg.org/image/an4vkk40j/)

### Proposed approach

One possible approach that builds on approach I is to utilize KMeans to cluster all pixels. Since the issue comes from the uncertain threshold value, we can potentially compute a proper threshold.  


## Data augmentation

The most common approach to secure training examples is to manually label some data. Recaptcha has various fonts with random scaling, translation, rotation and distortion. Assuming we need 1000 examples per class there are 36 classes, we may manually label 36000 data, which is too expensive. Thus, we design data augmentation process. Specifically, we can apply random scaling, translation, rotation and distortion, even interference lines, to all our labelled examples, as recaptcha did. By doing this, we drastically saved manual efforts.


## Pseudo labeled examples

Since we want to identify digits or letters, we even do not need to label examples at all. After examining the raw data, we could notice that there are some common fonts, including Highway Gothic, Filetto and KG Empire of Dirt. Therefore, we downloaded those fonts and generated examples using them.

[![generated_examples.png](https://s12.postimg.org/6xctn6ood/generated_examples.png)](https://postimg.org/image/bw0c1psh5/)

## Preliminary results
 
We first experimented fully-connected neural networks. We trained a model with 512 hidden neurons using the generated examples. We achieved a per-character error rate of 2.2% on test sets. However, so far, we are only able to classify images that are easy to slice, so we are still improving our slicing algorithm.

## End-to-end evaluation

The reCAPTCHA system contains blurry images even human find them difficult to read. This nature of reCAPTCHA system makes an evaluation based the correctness of text recognition trivial. For certain reCAPTCHA images, we do not have correct text to be compared against recognition results from our implementation. 
As the goal of this project is to assess the reCAPTCHA v1 system from a security perspective, ultimately, we want to evaluate the project by implementing an end-to-end system that aims to crack the bot detection system. We implemented a pipeline that retrieves a reCAPTCHA image, pre-processes the image, and performs text recognition by the model we trained before to construct response which will be sent back to Google. If the response passes Google authentication that means the recognition results is within Google’s fault tolerance designed for human user. 


